**Name:- Aditya Pawar**

**Class :- A (A1)**

**Roll No:- 04**

**PRN :- 22SC114501004**

**Experiment 2:- Impact of Data Quality on AI Fairness**

In [1]:
%pip install fairlearn

Note: you may need to restart the kernel to use updated packages.


In [2]:
%pip install ucimlrepo

Note: you may need to restart the kernel to use updated packages.


In [3]:
from ucimlrepo import fetch_ucirepo
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from fairlearn.preprocessing import CorrelationRemover
from fairlearn.metrics import MetricFrame, selection_rate, true_positive_rate, false_positive_rate, false_negative_rate


In [4]:
german_credit = fetch_ucirepo(id=144)

In [5]:
X = german_credit.data.features.copy()
y = german_credit.data.targets.copy()

In [6]:
df = pd.concat([X, y], axis=1)

In [7]:
print("First 5 rows of the dataset:")
display(df.head())

First 5 rows of the dataset:


,Attribute1,Attribute2,Attribute3,Attribute4,Attribute5,Attribute6,Attribute7,Attribute8,Attribute9,Attribute10,...,Attribute12,Attribute13,Attribute14,Attribute15,Attribute16,Attribute17,Attribute18,Attribute19,Attribute20,class
0,A11,6,A34,A43,1169,A65,A75,4,A93,A101,...,A121,67,A143,A152,2,A173,1,A192,A201,1
1,A12,48,A32,A43,5951,A61,A73,2,A92,A101,...,A121,22,A143,A152,1,A173,1,A191,A201,2
2,A14,12,A34,A46,2096,A61,A74,2,A93,A101,...,A121,49,A143,A152,1,A172,2,A191,A201,1
3,A11,42,A32,A42,7882,A61,A74,2,A93,A103,...,A122,45,A143,A153,1,A173,2,A191,A201,1
4,A11,24,A33,A40,4870,A61,A73,3,A93,A101,...,A124,53,A143,A153,2,A173,2,A191,A201,2


In [8]:
print("\nDataset shape (rows, columns):", df.shape)


Dataset shape (rows, columns): (1000, 21)


In [9]:
print("\nData types and non-null counts:")
print(df.info())


Data types and non-null counts:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 21 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Attribute1   1000 non-null   object
 1   Attribute2   1000 non-null   int64 
 2   Attribute3   1000 non-null   object
 3   Attribute4   1000 non-null   object
 4   Attribute5   1000 non-null   int64 
 5   Attribute6   1000 non-null   object
 6   Attribute7   1000 non-null   object
 7   Attribute8   1000 non-null   int64 
 8   Attribute9   1000 non-null   object
 9   Attribute10  1000 non-null   object
 10  Attribute11  1000 non-null   int64 
 11  Attribute12  1000 non-null   object
 12  Attribute13  1000 non-null   int64 
 13  Attribute14  1000 non-null   object
 14  Attribute15  1000 non-null   object
 15  Attribute16  1000 non-null   int64 
 16  Attribute17  1000 non-null   object
 17  Attribute18  1000 non-null   int64 
 18  Attribute19  1000 non-null   object


In [10]:

print("\nSummary statistics (numerical columns):")
display(df.describe())


Summary statistics (numerical columns):


,Attribute2,Attribute5,Attribute8,Attribute11,Attribute13,Attribute16,Attribute18,class
count,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000
mean,20.903000,3271.258000,2.973000,2.845000,35.546000,1.407000,1.155000,1.300000
std,12.058814,2822.736876,1.118715,1.103718,11.375469,0.577654,0.362086,0.458487
min,4.000000,250.000000,1.000000,1.000000,19.000000,1.000000,1.000000,1.000000
25%,12.000000,1365.500000,2.000000,2.000000,27.000000,1.000000,1.000000,1.000000
50%,18.000000,2319.500000,3.000000,3.000000,33.000000,1.000000,1.000000,1.000000
75%,24.000000,3972.250000,4.000000,4.000000,42.000000,2.000000,1.000000,2.000000
max,72.000000,18424.000000,4.000000,4.000000,75.000000,4.000000,2.000000,2.000000


In [11]:
print("\nTarget value counts:")
print(df[df.columns[-1]].value_counts())


Target value counts:
class
1    700
2    300
Name: count, dtype: int64


In [12]:
print("\nAny missing values?")
print(df.isnull().sum())


Any missing values?
Attribute1     0
Attribute2     0
Attribute3     0
Attribute4     0
Attribute5     0
Attribute6     0
Attribute7     0
Attribute8     0
Attribute9     0
Attribute10    0
Attribute11    0
Attribute12    0
Attribute13    0
Attribute14    0
Attribute15    0
Attribute16    0
Attribute17    0
Attribute18    0
Attribute19    0
Attribute20    0
class          0
dtype: int64


In [13]:
print("Shape of dataset:", X.shape)
print(X.head())
print(y.head())


Shape of dataset: (1000, 20)
  Attribute1  Attribute2 Attribute3 Attribute4  Attribute5 Attribute6  \
0        A11           6        A34        A43        1169        A65   
1        A12          48        A32        A43        5951        A61   
2        A14          12        A34        A46        2096        A61   
3        A11          42        A32        A42        7882        A61   
4        A11          24        A33        A40        4870        A61   

  Attribute7  Attribute8 Attribute9 Attribute10  Attribute11 Attribute12  \
0        A75           4        A93        A101            4        A121   
1        A73           2        A92        A101            2        A121   
2        A74           2        A93        A101            3        A121   
3        A74           2        A93        A103            4        A122   
4        A73           3        A93        A101            4        A124   

   Attribute13 Attribute14 Attribute15  Attribute16 Attribute17  Attribute1

In [14]:
label_encoders = {}
for col in X.columns:
    if X[col].dtype == 'object':
        le = LabelEncoder()
        X[col] = le.fit_transform(X[col])
        label_encoders[col] = le

In [15]:
scaler = StandardScaler()
X_scaled = pd.DataFrame(scaler.fit_transform(X), columns=X.columns)

In [16]:
sensitive_feature = "personal_status" if "personal_status" in X.columns else X.columns[0]
print("Sensitive feature chosen:", sensitive_feature)

Sensitive feature chosen: Attribute1


In [17]:
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size=0.2, random_state=42, stratify=y
)

In [18]:
A_train = X_train[sensitive_feature]
A_test = X_test[sensitive_feature]


In [19]:
model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)

c:\Users\adity\anaconda3\Lib\site-packages\sklearn\utils\validation.py:1408: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


LogisticRegression(max_iter=1000)

In [20]:
y_pred = model.predict(X_test)

acc_before = accuracy_score(y_test, y_pred)
print("Accuracy before mitigation:", acc_before)

Accuracy before mitigation: 0.765


In [21]:
# Convert y_test to a 1D array of integers
y_true = y_test["class"].values

# If the target labels are {1, 2}, set pos_label=1 for TPR, FPR, FNR
metrics_before = MetricFrame(
    metrics={
        'accuracy': accuracy_score,
        'selection_rate': selection_rate,
        'TPR': lambda y_true, y_pred: true_positive_rate(y_true, y_pred, pos_label=1),
        'FPR': lambda y_true, y_pred: false_positive_rate(y_true, y_pred, pos_label=1),
        'FNR': lambda y_true, y_pred: false_negative_rate(y_true, y_pred, pos_label=1)
    },
    y_true=y_true,
    y_pred=y_pred,
    sensitive_features=A_test
)

print("\nFairness Metrics Before Mitigation:")
print(metrics_before.by_group)
print("\nOverall Accuracy:", accuracy_score(y_test, y_pred))



Fairness Metrics Before Mitigation:
            accuracy  selection_rate       TPR       FPR       FNR
Attribute1                                                        
-1.254566   0.545455        0.400000  0.440000  0.366667  0.560000
-0.459026   0.763636        0.709091  0.882353  0.428571  0.117647
 0.336513   0.833333        0.916667  0.909091  1.000000  0.090909
 1.132053   0.910256        0.987179  1.000000  0.875000  0.000000

Overall Accuracy: 0.765


### Conclusion

The German Credit dataset is well-structured and complete, making it suitable for fairness evaluation experiments. However, the slight class imbalance and mixed data types mean that preprocessing steps such as scaling, encoding, and possibly rebalancing will be important. The next step will be to build a classification model and evaluate it using fairness metrics to understand any potential bias in predictions.